In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib notebook

# Introduction

To make operating with large and variable data more comfortable a number of special libraries have been developed. One important one is Pandas. Pandas is a big library with statistical tools, convenient data importing functions and tools to filter and extract data.

A whole bunch of tutorials for Pandas are available online:
https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html<br>
or<br>
https://dataanalysispython.readthedocs.io/en/latest/index.html<br>
The latter beeing my personal favorite<br>

We will focus on 3 functions of pandas:
1. The convenient way to slice by names
2. The import function, that has options for most types of data
3. The very simple but powerful plot function

Lets look on our Cookbook again, convert it into a DataFrame, the central storage format of Pandas and then instead of using the general print - function use the beautiful printing that is build into Pandas 

In [ ]:
import pandas as pd
Cookbook={'pancakes':{'description':'delicious food prepared fast',
                    'time':10.0,'portions':6},
        'chocolade icecream':{'description':'my favourite cholate icecream receipe',
                    'time':5.0,'portions':3}}
#Now we convert it to DataFrame and print it
df=pd.DataFrame(Cookbook)
df

the top horizontal bar (pancakes and chocolade icecream) is called columns and represent data-sets the vertical bar on the left is called index or rows and represents the content of each dataset. 

For categoric data like this cookbook we could now filter after how long it takes to make a receipe by horizontal (row) slicing

In [ ]:
df.loc['time',:]<9

or get the information for one dataset by vertical (column) slicing

In [ ]:
df.loc[:,'pancakes']

Here we have already used one of the great advantages of Pandas, the name based slicing using the "locator"<br>
df.loc["name of row","name of column"]<br>
As before we can use the ":" to represent "from beginning until" or "from here until", or simply "all"
This is where the 2 (or more!) dimensional access is really useful.

While this was kind of useful for categorical data, but becomes really useful for e.g. spectroscopic data<br>
Here we read in the measured absorption of silicon<br> 
and in the left panel slice only plot the visible range of the measured data<br> 
while in the right panel the series is plotted as read<br>

Important: Both the index and the columns are usually list of strings (text). To really use the clever slicing we need to convert them into floats (third line in code below)


In [ ]:
#read in the data and put the first column as index
df_spec=pd.read_csv("http://www.jensuhlig.de/Kemm30/Silicon_Absorption.csv",index_col=0)
df_spec.index=df_spec.index.astype(float)#convert the index from the 

fig,(ax_left,ax_right)=plt.subplots(1,2)  #ignore this for now, we are gone discuss plotting in the next notebook
df_spec.plot(ax=ax_right,title='As read')

df_sliced=df_spec[400.5:750.5]
df_sliced.plot(ax=ax_left,title='Sliced in the visible range')

Did you notice that we sliced from "400.5"? by looking into the original dataframe you will notice that the value 400.5 does not exist. The location based slicing can find the nearest position, if the type of the index are numbers. 

In [ ]:
df_spec[398:403]

You might have noticed that we used a df_spec[:] 1-dimension, while for the cookbook we used df.loc[:,'pancakes'] 2-dimensions

A single column DataFrame is called a series, which is its own class. However also a DataFrame can have only one column. (as df_spec). While this feels like a little bit confusing, this is originating in same difference as numpy arrays. 

In [ ]:
print(type(df),'has dimensions',df.ndim)
print(type(df.iloc[:,1]),'has dimensions',df.iloc[:,1].ndim)
print(type(df_spec),'has dimensions',df_spec.ndim)

This is also where a lot of the similarities come from. <br>
Pandas also has the usual index based slicing like in numpy using the "index locator"

In [ ]:
df_spec.iloc[10:13] # row 10 - 12

and all the usual math functions from numpy

In [ ]:
print('minimum',df_spec.min())
print('maximum',df_spec.max())
print('average',df_spec.mean())

and all the values can be converted back into numpy arrays (e.g. after importing) by using their "value"

In [ ]:
data_array=df_spec.values
index_array=df_spec.index.values
columns_array=df_spec.columns.values
#print(data_array,index_array,columns_array)

# Reading data with Pandas

### Task

Read the data in " http://www.jensuhlig.de/Kemm30/sinc.dat " using the function df=pd.read_csv() 

For this you can either open the file directly from the internet or download it into your working directory first. 

The open the file and note (maybe on a paper) 

1. if there is any text before the data that you do not want to read
2. if there are any headers (text before the data that tells what the columns are)
3. what are the separators between the numbers
4. which column (if any) would be good to use as index (think x-axis in plot)

Play with the options until you get the

in the index. the options filename (the internet adress), sep (the separator) and index_col (which column to use for the index) are the ones that you need to take a good look at. 

If everything worked and you named you imported dataframe *df_read* <br>
it should look like this if you call *df_read.head()*

<div>
<img src="http://www.jensuhlig.de/Kemm30/sinc_reading.jpg" width="600">
</div>

now try what df_read.plot() gives you

If you convert the index and the columns to numbers, sorting makes more sense. Compare the images

In [ ]:
df_read=df.copy() #to make a deep (real) copy
df_read.columns=df_read.columns.values.astype(float)
df_read.index=df_read.index.values.astype(float)
df_read.sort_index(inplace=True,axis='columns')
df_read.plot()

Lastly it is a good idea to give both columns and rows a name, which is then also used during plotting. The y-scale label you typically have to write by hand (tomorrow). Plot and compare

In [ ]:
df_read.index.name='x-value'
df_read.columns.name='parameter'

## Task

Produce the same plot for "http://www.jensuhlig.de/Kemm30/sinc_2.dat". The first row are the headers and should be read! if the data looks strange follow the stepwise check from above

### Dealing with directories

Before we start to load and handle many datafiles we should quickly look into how to handle directories. 
Loading a file from a sub directory needs the filesystem separator, meaning the sign that separates folders and files.

If you are working on "binder" or "garm" then the program of your Jupyter notebook is running on the server. This means that you need to upload your data to this server to be able to use it.

Lets assume that you have your datafiles in a (relative) subfolder with the name "data" Then during file import you need to replace you filename "sinc.dat" with the path to the file<br>
under windows this means using the backslash "data\sinc.dat"<br>
as however the "\" is a special character we need to use "\\" to create it

In [ ]:
filename='data\\sinc.dat' #under windows
filename='data/sinc.dat' #under MAC and Linux

The library "os" provides some simple tools to make this more comfortable <br>
"os.sep" provides the platform independent separator (on linux and windows alike) <br>
"os.getcwd()" read "get current working directory" can tell you where you currently are<br>
For more complicated path there is the excellent library "pathlib". 

In [ ]:
import os #import the library
filename='data' + os.sep + 'sinc.dat'
filename=os.sep.join(['data','sinc.dat']) #easy to read and independent of which platform

## Task

create a new folder in your working directory and copy the sinc.dat file into it (using the normal folders of the computer or the upload / new folder functions) . <br>
Read and plot this file

### Task

The data file " http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat " was measured at a large scale research facility: the "Advanced light source" in Chicago and represents X-ray absorption data. Ignore the first 30 rows (and header) and read all columns from the file. Hint the separator "\s+" separate for one or multiple white spaces.

Use copy paste to extract the column names (Row 30) from the text file and paste it in your Notebook as a long string. Then use the string method "split" to separate the string into a list of column names

give this list of names to the use the "names" parameter to give the columns in the DataFrame the right name. 

select the index column by the name "Energy" instead of the number.

Choose to only read the column "PR" (column 14)

If you have done the selection during the import then plotting the column "PR" vs "Energy" is simply
df_read.plot() 

If you import all columns, but want to select only one to plot you would use: df["PR"].plot()

If everything worked as expected the plot should look like this:
<div>
<img src="http://www.jensuhlig.de/Kemm30/APS_reading.jpg" width="300">
</div>    

Congratulation, this is most likely one of the most difficult files you ever need to read, combining all the techniques you have learned up to now.

# Creation of DataFrames

Dictionaries are often the easiest way to create and name DataFrames other methods are attaching columns to dataframes.

In [ ]:
timen=np.arange(0,60,0.1)   # create a time vector
dicten={}                   # create an empty dictionary to contain the data
for rate in np.arange(1.5,10,1):     # loop over the parameter you want to use and put the current parameter into "rate"
    y=np.exp(-timen/rate)            # create the vector with the y-values 
    dicten['%.1f'%rate]=y            # store the value in the dictionary with the parameter as the key 
df=pd.DataFrame(dicten)              # Now create the dataframe. 
df.index.name='time [s]'             # give the x-axis a name
df.columns.name='rate [mol/s]'       # give the parameter a name
df.name='Concentation'

## Task

plot the resulting data_frame

 For other ways please see the documentation and the cheatsheet. Usuful other methods are to concatenate multiple vectors or to simply calcualte new columns from old columns (the most used method)

# Advanced:

The manual solution to column reading would be:

In [ ]:
col_names='N  Epoch  Energy  k  Mono  Seconds  ion1  ion2  ion3  ion4  mud  mud1  mostabc  SYNM  PR  bun-1  bun-1GS  bun1  bun1GS  bun2  bun2GS  DifSYN  Difb_1  Difb1  Difb2  c0o0b0  c0o1b0  c0o2b0  c0o3b0  c0o4b0  c0o5b0  c0o6b0  c0o7b0  c0o8b0  c0o9b0  c0o10b0  c0o11b0  c0o12b0  c0o13b0  c0o14b0  c0o15b0  c0o16b0  c0o17b0  c0o18b0  c0o19b0  c0o20b0  c0o21b0  c1o0b0  c1o1b0  c1o2b0  c1o3b0  c1o4b0  c1o5b0  c1o6b0  c1o7b0  c1o8b0  c1o9b0  c1o10b0  c1o11b0  c1o12b0  c1o13b0  c1o14b0  c1o15b0  c1o16b0  c1o17b0  c1o18b0  c1o19b0  c1o20b0  c1o21b0  c2o0b0  c2o1b0  c2o2b0  c2o3b0  c2o4b0  c2o5b0  c2o6b0  c2o7b0  c2o8b0  c2o9b0  c2o10b0  c2o11b0  c2o12b0  c2o13b0  c2o14b0  c2o15b0  c2o16b0  c2o17b0  c2o18b0  c2o19b0  c2o20b0  c2o21b0  c3o0b0  c3o1b0  c3o2b0  c3o3b0  c3o4b0  c3o5b0  c3o6b0  c3o7b0  c3o8b0  c3o9b0  c3o10b0  c3o11b0  c3o12b0  c3o13b0  c3o14b0  c3o15b0  c3o16b0  c3o17b0  c3o18b0  c3o19b0  c3o20b0  c3o21b0'
col_names=col_names.split()
print('print the first 8 entries: {}'.format(col_names[:8]))
df3=pd.read_csv("http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat",skiprows=30,sep='\s+',names=col_names,index_col='Energy')

There are automatic ways how to do this column extraction:
The first method only reads the columns you want and reduces the work, but requires me to count

In [ ]:
df3a=pd.read_csv("http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat",skiprows=30,sep='\s+',names=['Energy','PR'],index_col=0,usecols=[2,14])

The second method uses an escape character to extract the column names automatically and allows then the use of the name:

In [ ]:
df3b=pd.read_csv("http://www.jensuhlig.de/Kemm30/APS_Copper_SolarCell.dat",skiprows=29,escapechar='L',sep='\s+',index_col=0,usecols=['Energy','PR'])

### Advanced Task: 

Create the a DataFrame with position from -5 cm to 5 cm as the index and in the columns different gaussians bell curves <br>
${\frac {1}{\sqrt {2\pi \sigma ^{2}}}}\operatorname {exp} \left(-{\frac {\left(x-\mu \right)^{2}}{2\sigma ^{2}}}\right)$<br>
with the same central position (mu=0) and different width sigma (0.5,1,2,3,4)
using the simplified plotting from above to show them in the same plot

# Pandas (much more about this later)

In the previous exercise you _manually_ extracted data from the internet which is problematic due to several reasons:

1. it's time consuming
2. it's error prone
3. what if the source is updated / corrected?

The following example uses _Pandas_ which is an external module used to handle large data bases; millions of entries is not a problem. As you will see, we simple point it to the Wikipedia page and it will automatically - and almost magically - detect the table and extract the values.

In [ ]:
import pandas as pd
tables = pd.read_html("https://en.wikipedia.org/wiki/Hydrophobicity_scales")
p = tables[0] # list of table found. Only one is found on the page. 
p.head() # show the first five rows (the head)

In [ ]:
p.to_excel('hydrophobicity.xlsx') # save to MS Excel file

In [ ]:
# we "zip" up two columns to form a dictionary. In Pandas, index can be given by their names.
d = dict( zip( p['Amino acid'], p['Interface scale, ΔGwif (kcal/mol)'] ) )
d

In [ ]:
d['Cys']

In [ ]:
# let's do a simple plot using Matplotlib (imported at the top). More on this later!
delta_G_values = list(d.values())
delta_G_values = [float(i.replace('−', '-')) for i in delta_G_values] # weird minus sign on wikipedia!
aminoacid_names = list(d.keys())
plt.bar(aminoacid_names, delta_G_values) # bar plot expects x and y values as lists
plt.xticks(rotation=90)
plt.xlabel('amino acid')
plt.ylabel('$\Delta G$ (kcal/mol)')
plt.title('source: wikipedia')
plt.show()

### Bonus Exercise

In the above we created two lists `delta_G_values` and `aminoacid_names` but the data is taken directly from the Wikipedia table and unsorted with respect to $\Delta G$. Use the answer to [this question](https://stackoverflow.com/questions/9764298/is-it-possible-to-sort-two-listswhich-reference-each-other-in-the-exact-same-w) to simultaneously sort both lists and re-plot the results.

### Excercise

1. Extract all tables from Wikipedias [list of Nobel laureates in Chemistry](https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Chemistry) into a list named `nobel`
1. How many tables did you find?
1. Show and investigate the first table
1. Try the following code:
~~~ py
# https://stackoverflow.com/questions/40581312/how-to-create-a-frequency-table-in-pandas-python
df = pd.value_counts(nobel[0]['Country[B]']).to_frame().reset_index()
mask = df['Country[B]']>1 # only countries with two or more prizes
df = df[mask]
explode = df['index']=='Sweden'
plt.pie( df['Country[B]'].values, labels=df['index'], autopct='%1.0f%%', radius=1.5, explode=explode )
plt.show()
~~~
1. have a look at `df`.
1. have a look at `mask`. How does it work?
1. have a look at `explode`. How does it work?


In [ ]:
url     = 'https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Chemistry'
nobel   = pd.read_html(url)
df      = pd.value_counts(nobel[0]['Country[B]']).to_frame().reset_index()
mask    = df['Country[B]']>1 # only countries with two or more prizes
df      = df[mask]
explode = df['index']=='Sweden'
plt.pie(df['Country[B]'].values,
        labels=df['index'],
        autopct='%1.0f%%',
        radius=1., 
        explode=explode )
plt.show()